# Building and Evaluating a Hidden Markov Model and a Viterbi Algorithm in NLP

## By Brea Koenes

### Overview
Build and evaluate a Hidden Markov Model (HMM) with a Viterbi algorithm in the field of Natural Language Processing (NLP). Use the Brown corpus from the NLTK library, focusing on the categories 'news', 'editorial', and 'reviews' with a 'universal' tagset. The purpose is to implement these fundamental concepts in NLP and to understand their applications and limitations.


## Preparing the Environment

Import the necessary libraries. I use NLTK for accessing linguistic data and algorithms, including the Brown corpus, and `train_test_split` from `sklearn.model_selection` for splitting data. The Brown Corpus was the first million-word electronic corpus of English, created in 1961 at Brown University.

This corpus contains text from 500 sources, and the sources have been categorized by genre, such as news, editorial, and so on.

In [2]:
import nltk
from nltk.corpus import brown
from sklearn.model_selection import train_test_split

nltk.download('brown')
nltk.download('universal_tagset')


## Loading and Exploring the Data

Load the 'Brown' corpus, focusing on specific categories: 'news', 'editorial', and 'reviews'. We'll use the 'universal' tagset for a more generalizable analysis. Utilize `brown.tagged_sents(categories=['news', 'editorial', 'reviews'], tagset='universal')` to load the data.

The **universal** tagset is a simplified schema developed to facilitate the comparison of grammatical categories across different languages. This tagset includes categories like:

    NOUN (noun)
    VERB (verb)
    ADJ (adjective)
    ADV (adverb)
    PRON (pronoun)
    DET (determiner, includes articles and quantifiers)
    ADP (adposition, includes prepositions and postpositions)
    NUM (numeral)
    CONJ (conjunction)
    PRT (particle, includes small function words like 'to' that are not clearly categorized under the above)
    . (punctuation)
    X (other category, including undefined and erroneous cases)

The `tagged_sents()` returns a list comprised of sentences, where each sentence is another list of word-tag pairs. Each pair consists of a word from the sentence and its corresponding part-of-speech tag.

In [9]:
brown.tagged_sents(categories=['news', 'editorial', 'reviews'], tagset='universal')

brown_tagged_sents = brown.tagged_sents(categories=['news', 'editorial', 'reviews'], tagset='universal')

In [10]:
brown_tagged_sents

[[('The', 'DET'), ('Fulton', 'NOUN'), ('County', 'NOUN'), ('Grand', 'ADJ'), ('Jury', 'NOUN'), ('said', 'VERB'), ('Friday', 'NOUN'), ('an', 'DET'), ('investigation', 'NOUN'), ('of', 'ADP'), ("Atlanta's", 'NOUN'), ('recent', 'ADJ'), ('primary', 'NOUN'), ('election', 'NOUN'), ('produced', 'VERB'), ('``', '.'), ('no', 'DET'), ('evidence', 'NOUN'), ("''", '.'), ('that', 'ADP'), ('any', 'DET'), ('irregularities', 'NOUN'), ('took', 'VERB'), ('place', 'NOUN'), ('.', '.')], [('The', 'DET'), ('jury', 'NOUN'), ('further', 'ADV'), ('said', 'VERB'), ('in', 'ADP'), ('term-end', 'NOUN'), ('presentments', 'NOUN'), ('that', 'ADP'), ('the', 'DET'), ('City', 'NOUN'), ('Executive', 'ADJ'), ('Committee', 'NOUN'), (',', '.'), ('which', 'DET'), ('had', 'VERB'), ('over-all', 'ADJ'), ('charge', 'NOUN'), ('of', 'ADP'), ('the', 'DET'), ('election', 'NOUN'), (',', '.'), ('``', '.'), ('deserves', 'VERB'), ('the', 'DET'), ('praise', 'NOUN'), ('and', 'CONJ'), ('thanks', 'NOUN'), ('of', 'ADP'), ('the', 'DET'), ('City


## Data Preprocessing

For the Hidden Markov Model, preprocess the data to ensure consistency and effectiveness. Apply lowercasing to each word in the (word, tag) tuples in our dataset.

In [11]:
brown_tagged_sents_lower = [
    [(word.lower(),tag) for word, tag in sentence]
    for sentence in brown_tagged_sents]

brown_tagged_sents_lower[:1]

[[('the', 'DET'),
  ('fulton', 'NOUN'),
  ('county', 'NOUN'),
  ('grand', 'ADJ'),
  ('jury', 'NOUN'),
  ('said', 'VERB'),
  ('friday', 'NOUN'),
  ('an', 'DET'),
  ('investigation', 'NOUN'),
  ('of', 'ADP'),
  ("atlanta's", 'NOUN'),
  ('recent', 'ADJ'),
  ('primary', 'NOUN'),
  ('election', 'NOUN'),
  ('produced', 'VERB'),
  ('``', '.'),
  ('no', 'DET'),
  ('evidence', 'NOUN'),
  ("''", '.'),
  ('that', 'ADP'),
  ('any', 'DET'),
  ('irregularities', 'NOUN'),
  ('took', 'VERB'),
  ('place', 'NOUN'),
  ('.', '.')]]

## Split Train and Test

Using train_test_split from sklearn, split the dataset from the previous step into train and test sets.



In [13]:
train_sents, test_sents = train_test_split(brown_tagged_sents_lower,
                                           test_size=0.2, # 80/20 split
                                           random_state=42
                                          )


## Training the Hidden Markov Model with Viterbi

Construct a dictionary of words in the form of a python list that includes every unique word found in the training set. Follow the same process for the tags set.

In [14]:
# Extract words and tags from training set
train_words = [word for sentence in train_sents for word, _ in sentence]
train_tags = [tag for sentence in train_sents for _, tag in sentence]

# Create vobab of unique words and tags
vocabulary = list(set(train_words))
tag_set = list(set(train_tags))

For building the Hidden Markov Model (HMM), utilize the [`HiddenMarkovModelTrainer`](https://tedboy.github.io/nlps/generated/generated/nltk.HiddenMarkovModelTrainer.html) class from NLTK.  

This class encapsulates [**both the HMM and the Viterbi algorithm**](https://www.nltk.org/api/nltk.tag.hmm.html). The Viterbi algorithm is used here to determine the most likely sequence of tags (states) for a given sequence of words (observations), based on the probabilities learned by the HMM. It's essential to import the `HiddenMarkovModelTrainer` from nltk.tag for this purpose.

Create an object from the `HiddenMarkovModelTrainer` using the tag set list and the dictionary list created before. This object will be used to train our HMM models.

Train five different HMM models:

- A 'pure' HMM without smoothing (For more about Smoothing, read chapter three of this [thesis](https://digitalscholarship.unlv.edu/cgi/viewcontent.cgi?article=2008&context=thesesdissertations#:~:text=Smoothing%20techniques%20in%20HMM%20will,to%20produce%20more%20accurate%20probabilities.))

    Train a model using the HiddenMarkovModelTrainer object (train_supervised method), passing only the 'train' dataset
    
    
- HMM with [LidstoneProbDist](https://en.wikipedia.org/wiki/Additive_smoothing) smoothing and gamma = 0.01

     LidstoneProbDist is an implementation of a Lidstone probability distribution, which is a variant of the Laplace distribution. The gamma parameter is the smoothing factor. The value of gamma determines the degree of smoothing applied. It's generally a positive number. A gamma of 1 corresponds to Laplace smoothing (add-one), while values different from 1 indicate different degrees of smoothing
     
     Train a model using the HiddenMarkovModelTrainer object (train_supervised method), passing the 'train' dataset and
     supplied function `lidstone_prob_dist_001`
     
     
- HMM with [LidstoneProbDist](https://en.wikipedia.org/wiki/Additive_smoothing) smoothing and gamma = 0.1
    
    Same as above, but with different gamma value.

    Train a model using the HiddenMarkovModelTrainer object (train_supervised method), passing the 'train' dataset and the suplied function `lidstone_prob_dist_01`
    
    
- HMM with [MLEProbDist (Maximum Likelihood Estimation)](https://en.wikipedia.org/wiki/Maximum_likelihood_estimation)

    The basic idea of MLE is to choose the parameters of a model in such a way that the likelihood (probability) of the observed data is maximized. In other words, MLE seeks the parameter values that make the observed data most probable.
     
    Train a model using the HiddenMarkovModelTrainer object (train_supervised method), passing the 'train' dataset and  the suplied function `MLE_ProbDist`
    
     
- HMM with [ELEProbDist (Expected Likelihood Estimation)](https://machinelearningmastery.com/what-is-maximum-likelihood-estimation-in-machine-learning/)

    This method is a form of statistical smoothing, similar to LidstoneProbDist and LaplaceProbDist, but with a slightly different approach.

    The idea behind ELE smoothing is to adjust probabilities in a way that balances accuracy in modeling frequently occurring events with the capability to handle rare or unobserved events. In simple terms, ELE smoothing attempts to estimate the probability of future events based on observed frequency, making adjustments to ensure that unobserved events are not given a probability of zero.
     
    Train a model using the HiddenMarkovModelTrainer object (train_supervised method), passing the 'train' dataset and the suplied function `ELE_ProbDist`

In [15]:
from nltk.probability import LidstoneProbDist, MLEProbDist, ELEProbDist

def lidstone_prob_dist_001(fd, bins):
    return LidstoneProbDist(fd, 0.01)

def lidstone_prob_dist_01(fd, bins):
    return LidstoneProbDist(fd, 0.1)

def MLE_ProbDist(fd, bins):
    return MLEProbDist(fd)

def ELE_ProbDist(fd, bins):
    return ELEProbDist(fd)

In [17]:
# Importing necessary modules from NLTK for HMM training and probability distributions
from nltk.tag import HiddenMarkovModelTrainer
from nltk.probability import LidstoneProbDist, MLEProbDist, ELEProbDist

# Create trainer
trainer = HiddenMarkovModelTrainer(states=tag_set,symbols=vocabulary)

# Model 1
hmm_pure = trainer.train_supervised(train_sents)

# Model 2
hmm_lidstone_001 = trainer.train_supervised(train_sents, estimator=lidstone_prob_dist_001)

# Model 3
hmm_lidstone_01 = trainer.train_supervised(train_sents, estimator=lidstone_prob_dist_01)

# Model 4
hmm_mle = trainer.train_supervised(train_sents,estimator=MLE_ProbDist)

# Model 5
hmm_ele = trainer.train_supervised(train_sents,estimator=ELE_ProbDist)

## Applying the HMM with Viterbi Algorithm

Predict the tags from the 'test' dataset using **each of the models created before**.

Use the `best_path` (docs [here](https://www.nltk.org/api/nltk.tag.hmm.html#nltk.tag.hmm.HiddenMarkovModelTagger.best_path)) function from the model. This function is used to predict the most likely sequence of tags for the given sequence of words. The `best_path` takes an unlabelled (without tags) sentence and returns a sequence of predicted tags.

Make sure to 'break' the 'test' dataset and use only the sentence (without tags) part.

In [18]:
# Break the test dataset to use only the sentences (words) without tags
test_sentences = [[word for word, tag in sentence] for sentence in test_sents]

# Predict tags for the test dataset using each model
# Model 1: Pure HMM
print("\nPredicting tags with Pure HMM...")
predicted_pure = [hmm_pure.tag(sentence) for sentence in test_sentences]
print("Prediction with Pure HMM completed.")

# Model 2: HMM with Lidstone smoothing (gamma=0.01)
print("\nPredicting tags with HMM (Lidstone, gamma=0.01)...")
predicted_lidstone_001 = [hmm_lidstone_001.tag(sentence) for sentence in test_sentences]
print("Prediction with HMM (Lidstone, gamma=0.01) completed.")

# Model 3: HMM with Lidstone smoothing (gamma=0.1)
print("\nPredicting tags with HMM (Lidstone, gamma=0.1)...")
predicted_lidstone_01 = [hmm_lidstone_01.tag(sentence) for sentence in test_sentences]
print("Prediction with HMM (Lidstone, gamma=0.1) completed.")

# Model 4: HMM with MLEProbDist
print("\nPredicting tags with HMM (MLEProbDist)...")
predicted_mle = [hmm_mle.tag(sentence) for sentence in test_sentences]
print("Prediction with HMM (MLEProbDist) completed.")

# Model 5: HMM with ELEProbDist
print("\nPredicting tags with HMM (ELEProbDist)...")
predicted_ele = [hmm_ele.tag(sentence) for sentence in test_sentences]
print("Prediction with HMM (ELEProbDist) completed.")


Predicting tags with Pure HMM...


/opt/anaconda3/lib/python3.12/site-packages/nltk/tag/hmm.py:335: RuntimeWarning: overflow encountered in cast
  O[i, k] = self._output_logprob(si, self._symbols[k])
/opt/anaconda3/lib/python3.12/site-packages/nltk/tag/hmm.py:333: RuntimeWarning: overflow encountered in cast
  X[i, j] = self._transitions[si].logprob(self._states[j])
/opt/anaconda3/lib/python3.12/site-packages/nltk/tag/hmm.py:363: RuntimeWarning: overflow encountered in cast
  O[i, k] = self._output_logprob(si, self._symbols[k])


Prediction with Pure HMM completed.

Predicting tags with HMM (Lidstone, gamma=0.01)...
Prediction with HMM (Lidstone, gamma=0.01) completed.

Predicting tags with HMM (Lidstone, gamma=0.1)...
Prediction with HMM (Lidstone, gamma=0.1) completed.

Predicting tags with HMM (MLEProbDist)...
Prediction with HMM (MLEProbDist) completed.

Predicting tags with HMM (ELEProbDist)...
Prediction with HMM (ELEProbDist) completed.



## Model Evaluation

Evaluate the performance of our HMM equipped with the Viterbi algorithm to gauge how effectively it handles unseen data. This involves comparing the tags predicted by our model on the test dataset against the actual tags.

* Compute Precision, Recall, and F1-score for each tag and the overall model.

* Print a confusion matrix that provides insights into the types of errors made by the model and helps in evaluating the accuracy of predictions

In [20]:
from nltk.metrics import ConfusionMatrix
import itertools

def printConfusionMatrix(labels_predicted, labels_correct):
    actual_tags = list(itertools.chain(*[[tag for word, tag in sent] for sent in labels_correct]))
    predicted_tags = list(itertools.chain(*[[tag for word, tag in sent] for sent in labels_predicted]))
    conf_matrix = ConfusionMatrix(actual_tags, predicted_tags)
    print(conf_matrix)

In [21]:
# Importing necessary libraries for model evaluation and metrics calculation
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelBinarizer
from itertools import chain
import numpy as np

# Defining the conlleval function for evaluating NLP models
def printConlleval(labels_predicted, labels_correct):
    lb = LabelBinarizer() # Initializing the LabelBinarizer for handling label encoding

    # Flattening the list of labels for correct and predicted
    labels_correct_flattened = [(word, tag) for sent in labels_correct for word, tag in sent]
    labels_predicted_flattened = [(word, tag) for sent in labels_predicted for word, tag in list(sent)]

    # Transforming the labels into a binary format for evaluation
    y_true_combined = lb.fit_transform([tag for _, tag in labels_correct_flattened])
    y_pred_combined = lb.transform([tag for _, tag in labels_predicted_flattened])

    tagset = set(lb.classes_)
    tagset = sorted(tagset, key=lambda tag: tag.split('-', 1)[::-1])
    class_indices = {cls: idx for idx, cls in enumerate(lb.classes_)}

    num_sentences = len(labels_predicted)
    total_tokens = sum(len(s) for s in labels_predicted)

    num_correct_sentences, total_correct_tokens = 0, 0
    for pred, true in zip(labels_predicted, labels_correct):
        if len(pred) == len(true):
            correct_tokens = sum(p == t for p, t in zip(pred, true))
            total_correct_tokens += correct_tokens
            if correct_tokens == len(pred):
                num_correct_sentences += 1

    correct_sentences_percentage = num_correct_sentences / num_sentences * 100
    total_correct_tokens_percentage = total_correct_tokens / total_tokens * 100

    classification_report_dict = classification_report(
        y_true_combined,
        y_pred_combined,
        labels=[class_indices[cls] for cls in tagset],
        target_names=tagset,
        output_dict=True,
        zero_division=1
    )


    classification_report_dict.pop('macro avg', None)
    classification_report_dict.pop('weighted avg', None)
    classification_report_dict.pop('samples avg', None)
    classification_report_dict.pop('micro avg', None)

    total_precision = precision_score(y_true_combined, y_pred_combined, average='weighted', zero_division=1)
    total_recall = recall_score(y_true_combined, y_pred_combined, average='weighted', zero_division=1)
    total_f1 = f1_score(y_true_combined, y_pred_combined, average='weighted', zero_division=1)
    total_line = f"{'Total':<15s} {total_precision:<10.2f} {total_recall:<10.2f} {total_f1:<10.2f}"

    report_lines = [f"{k:<15s} {classification_report_dict[k]['precision']:<10.2f} {classification_report_dict[k]['recall']:<10.2f} {classification_report_dict[k]['f1-score']:<10.2f}" for k in classification_report_dict if isinstance(classification_report_dict[k], dict)]
    report_lines.insert(0, "\n")
    report_lines.insert(1, f"{'TAG':<15s} {'Precision':<10s} {'Recall':<10s} {'F1-score':<10s}\n")
    report_lines.insert(2, total_line)
    report_lines.insert(3, '-'*50 + '\n')
    classification_report_str = "\n".join(report_lines)

    additional_info_str = ''
    additional_info_str += f'Total tokens: {total_tokens}\n'
    additional_info_str += f'Total correct tokens: {total_correct_tokens} ({total_correct_tokens_percentage:.2f}%)\n'
    additional_info_str += f'Processed sentences: {num_sentences}\n'
    additional_info_str += f'Completely correct sentences: {num_correct_sentences} ({correct_sentences_percentage:.2f}%)\n'

    print(additional_info_str + classification_report_str)

In [22]:
# The correct labels are the tags from test_sents
labels_correct = test_sents

# Evaluate each model using printContLevel and printConfusionMatrix
# Model 1: Pure HMM
print("\n=== Evaluation for Pure HMM ===")
printConlleval(predicted_pure, labels_correct)
printConfusionMatrix(predicted_pure, labels_correct)

# Model 2: HMM with Lidstone smoothing (gamma=0.01)
print("\n=== Evaluation for HMM (Lidstone, gamma=0.01) ===")
printConlleval(predicted_lidstone_001, labels_correct)
printConfusionMatrix(predicted_lidstone_001, labels_correct)

# Model 3: HMM with Lidstone smoothing (gamma=0.1)
print("\n=== Evaluation for HMM (Lidstone, gamma=0.1) ===")
printConlleval(predicted_lidstone_01, labels_correct)
printConfusionMatrix(predicted_lidstone_01, labels_correct)

# Model 4: HMM with MLEProbDist
print("\n=== Evaluation for HMM (MLEProbDist) ===")
printConlleval(predicted_mle, labels_correct)
printConfusionMatrix(predicted_mle, labels_correct)

# Model 5: HMM with ELEProbDist
print("\n=== Evaluation for HMM (ELEProbDist) ===")
printConlleval(predicted_ele, labels_correct)
printConfusionMatrix(predicted_ele, labels_correct)


=== Evaluation for Pure HMM ===
Total tokens: 40434
Total correct tokens: 23277 (57.57%)
Processed sentences: 1875
Completely correct sentences: 489 (26.08%)


TAG             Precision  Recall     F1-score  

Total           0.86       0.58       0.63      
--------------------------------------------------

.               1.00       0.45       0.62      
ADJ             0.93       0.46       0.62      
ADP             0.97       0.51       0.67      
ADV             0.89       0.54       0.68      
CONJ            0.99       0.48       0.65      
DET             1.00       0.59       0.74      
NOUN            0.97       0.47       0.64      
NUM             0.98       0.53       0.69      
PRON            0.96       0.67       0.79      
PRT             0.88       0.53       0.67      
VERB            0.26       0.98       0.41      
X               1.00       0.28       0.43      
     |                        C         N         P         V      |
     |         A    A    A    O

## Choose the best model

Choose the best model and export it.

In [49]:
# Importing dill library for model serialization
import dill
mybestmodel = hmm_lidstone_001

# serialization with dill
with open('mybestmodel.dill', 'wb') as file:
    dill.dump(mybestmodel, file)